In [39]:
import os
import pandas as pd
import re
import json
from itertools import combinations

# cambiar url para leer los datos de otro subfolder con path relativo
# os.chdir('../')

# Importar stop words en espanol
from nltk.corpus import stopwords
spanish_stop_words = set(stopwords.words('spanish'))

In [ ]:
# Diccionario de siglas a eliminar dentro del nombre del cliente
# Se ejecuta solo una vez solo si se requiere actualizar la lista de siglas a elimnar

# lista_companias_filtro = ["CORP",'PLC','EPS','CI','SCA','INC','P.A.C','GROUP','SA','SAS','LTDA','LTD','LIMITED','ESP','SOCIEDAD','LP','BV','BIC']
# diccionario = {}
# diccionario["siglas"] = [word.lower() for word in lista_companias_filtro]

# with open("data/archivos_auxiliares/diccionario_siglas.json","w") as f:
#     json.dump(diccionario, f)

In [ ]:
# importacion de datos
noticias = pd.read_csv('../datos/noticias.csv')
clientes = pd.read_csv('../datos/clientes.csv')
clientes_noticias = pd.read_csv('../datos/clientes_noticias.csv')
with open("data/archivos_auxiliares/diccionario_siglas.json", 'r') as f:
    diccionario_siglas = json.load(f)
    lista_siglas = diccionario_siglas['siglas']

In [168]:
# funciones utiles
from distutils.command.clean import clean

def check_cliente(cliente, noticia):

    cliente_edit = cliente.split(" ")

    if len(cliente_edit)<3:

        condicion = bool(re.findall(cliente,noticia))
        if condicion:
            return 'Cliente'
        else:
            return 'No Aplica'        
    else:
        combs = [ " ".join(list(x)) for x in combinations(cliente_edit, 2)]
        condicion = [bool(re.findall(x,noticia)) for x in combs]

        if True in condicion:
            return "Cliente_"
            
        else:
            return "no"

def clean_name(text,lista_filtro):

    """
    Esta funcion limpia una cadena de texto excluendo todas aquellas palabras que aparecen dentro de lista_filtro.
    Se usa por defecto la separación de la cadena con espacios en blanco.

    Inputs: 

        text : str

            Texto a limpiar
        
        lista_filtro : list

            Lista de palabras a remover
        
    Outputs:

        String con la cadena de texto limpia

    """
    lista_nombre = text.split(' ')
    nombre_final = [word for word in lista_nombre if word not in lista_filtro and word not in spanish_stop_words]
    return ' '.join(nombre_final)

def elimina_letras_sueltas(text):
    
    text_clean = text.split(" ")
    text_clean = [word for word in text_clean if len(word)>1]

    return " ".join(text_clean)


def clean_df(df, *args, **kwards):

    df_clean = df.copy()
    df_clean['nombre_clean'] = df_clean['nombre'].apply(lambda x: x.replace('.',''))
    df_clean['nombre_clean'] = df_clean['nombre_clean'].apply(lambda x: clean_name(text = x.lower(),*args, **kwards))
    df_clean['nombre_clean'] = df_clean['nombre_clean'].apply(elimina_letras_sueltas)
    df_clean['long_nombre'] = df_clean['nombre_clean'].apply(lambda x: len(x))
    df_clean['palabras_nombre'] = df_clean['nombre_clean'].apply(lambda x: len(x.split(' ')))
    df_clean = df_clean[['nit',"nombre_clean"]]

    return df_clean

def preprocessing_noticas_clientes(x):
    x['Participacion'] = x.apply(lambda x: check_cliente(cliente=x['nombre_clean'], noticia=x['news_text_content']))    
    return x

def noticia_cliente_detalle(noticias,clientes_noticias):
    final_df = clientes_noticias.merge(noticias, on = 'news_id', how='left')
    final_df = final_df[['nit','news_id', 'news_text_content']]
    return final_df

In [169]:
# Limpieza de dfs
clientes_clean = clean_df(clientes, lista_filtro = lista_siglas)
noticia_cliente_detalle_df = noticia_cliente_detalle(noticias,clientes_noticias)
participacion_df = clientes_clean.merge(noticia_cliente_detalle_df, on = 'nit', how = 'left')
participacion_df["Participacion"] = participacion_df.apply(lambda x: check_cliente(cliente=x['nombre_clean'], noticia=x['news_text_content']), axis=1)    


In [177]:
participacion_df['Participacion'].value_counts()

No Aplica    40337
no           32314
Cliente       1445
Cliente_       521
Name: Participacion, dtype: int64

In [172]:
participacion_df[participacion_df['Participacion']=='Cliente_'].head(40)

,nit,nombre_clean,news_id,news_text_content,Participacion
593,860005264,fabrica grasas productos quimicos,news14869,El sector servicios elevo su facturacion un 1...,Cliente_
605,860005264,fabrica grasas productos quimicos,news56806,La Organizacion Mundial de la Salud (OMS) ha ...,Cliente_
607,860005264,fabrica grasas productos quimicos,news58983,La diversificacion de la industria manufacture...,Cliente_
609,860005264,fabrica grasas productos quimicos,news94301,QUITO -- Pragmatismo y pensamiento tecnico ser...,Cliente_
612,860005264,fabrica grasas productos quimicos,news94341,La semana pasada el DANE presento al pais las ...,Cliente_
615,860005264,fabrica grasas productos quimicos,news94348,Segun los ultimos datos de comercio declarado...,Cliente_
785,860401826,productos alimenticios recreo,news21435,Las manufacturas hicieron parte de los sectore...,Cliente_
788,860401826,productos alimenticios recreo,news58983,La diversificacion de la industria manufacture...,Cliente_
799,860401826,productos alimenticios recreo,news97630,La guerra entre Rusia y Ucrania ya lleva casi ...,Cliente_
800,860401826,productos alimenticios recreo,news97631,Les contaba en la pasada columna como varios p...,Cliente_


In [176]:
noticias[noticias['news_id']=='news11843'].loc[473,'news_url_absolute']

'https://www.semana.com/economia/macroeconomia/articulo/canasta-familiar-cuales-productos-suben-y-cuales-bajan-de-precio/202216/'

In [159]:
clientes['nombre_editado'] = clientes['nombre'].str.lower()
clientes['val_nombre_editado'] = clientes['nombre_editado'].apply(lambda x: bool(re.findall('davivienda',x)))
clientes[clientes['val_nombre_editado']]
noticias['nombre_editado'] = noticias['news_text_content'].str.lower()
noticias['val_nombre_editado'] = noticias['nombre_editado'].apply(lambda x: bool(re.findall('eps',x)))
davi = noticias[noticias['val_nombre_editado']].copy()
davi = davi.merge(clientes_noticias, on = 'news_id', how = 'left')
davi.loc[1,'news_text_content']
clientes[clientes['nit'].isin(list(davi['nit'].unique()))]
x = davi[davi['nit']==805027024].copy().reset_index()
x.loc[0,'news_url_absolute_x'] # news_url_absolute_x, news_title, news_text_content	